# config

> Configuring nbdev and bootstrapping notebook export


In [ ]:
#|default_exp config

In [ ]:
#|export
_doc_ = """Read and write nbdev's `settings.ini` file.
`get_config` is the main function for reading settings."""

In [ ]:
#|export
from datetime import datetime
from fastcore.docments import *
from fastcore.utils import *
from fastcore.meta import *
from fastcore.script import *
from fastcore.style import *
from fastcore.xdg import *

import ast
from IPython.display import Markdown
from execnb.nbio import read_nb,NbCell
from urllib.error import HTTPError

In [ ]:
#|hide
from fastcore.test import *
import tempfile

## Configuring nbdev

nbdev is heavily customizeable, thanks to the configuration system defined in this module. There are 2 ways to interact with nbdev's config:

- **In the terminal:** `nbdev_create_config` creates a config file (if you're starting a new project use `nbdev_new` instead)
- **In your library:** `get_config` returns a `fastcore.foundation.Config` object.

Read on for more about how these work.

### Creating a config file -

In [ ]:
#|export
_nbdev_home_dir = 'nbdev' # sub-directory of xdg base dir
_nbdev_cfg_name = 'settings.ini'

In [ ]:
#|export
def _git_repo():
    try: return repo_details(run('git config --get remote.origin.url'))[1]
    except OSError: return

In [ ]:
#|hide
test_eq(_git_repo(), 'UAV')
with tempfile.TemporaryDirectory() as d, working_directory(d): test_is(_git_repo(), None)

In [ ]:
#|export

# When adding a named default to the list below, be sure that that name
# is also added to one of the sections in `_nbdev_cfg_sections` as well,
# or it won't get written by `nbdev_create_config`:
def _apply_defaults(
    cfg,
    lib_name='%(repo)s', # Package name
    git_url='https://github.com/%(user)s/%(repo)s', # Repo URL
    custom_sidebar:bool_arg=False, # Use a custom sidebar.yml?
    nbs_path:Path='nbs', # Path to notebooks
    lib_path:Path=None, # Path to package root (default: `repo` with `-` replaced by `_`)
    doc_path:Path='_docs', # Path to rendered docs
    tst_flags='notest', # Test flags
    version='0.0.1', # Version of this release
    doc_host='https://%(user)s.github.io',  # Hostname for docs
    doc_baseurl='/%(repo)s',  # Base URL for docs
    keywords='nbdev jupyter notebook python', # Package keywords
    license='apache2', # License for the package
    copyright:str=None, # Copyright for the package, defaults to '`current_year` onwards, `author`'
    status='3', # Development status PyPI classifier
    min_python='3.7', # Minimum Python version PyPI classifier
    audience='Developers', # Intended audience PyPI classifier
    language='English', # Language PyPI classifier
    recursive:bool_arg=True, # Include subfolders in notebook globs?
    black_formatting:bool_arg=False, # Format libraries with black?
    readme_nb='index.ipynb', # Notebook to export as repo readme
    title='%(lib_name)s', # Quarto website title
    allowed_metadata_keys='', # Preserve the list of keys in the main notebook metadata
    allowed_cell_metadata_keys='', # Preserve the list of keys in cell level metadata
    jupyter_hooks:bool_arg=True, # Run Jupyter hooks?
    clean_ids:bool_arg=True, # Remove ids from plaintext reprs?
    clear_all:bool_arg=False, # Remove all cell metadata and cell outputs?
    put_version_in_init:bool_arg=True, # Add the version to the main __init__.py in nbdev_export
):
    "Apply default settings where missing in `cfg`."
    if getattr(cfg,'repo',None) is None:
        cfg.repo = _git_repo()
        if cfg.repo is None:
            _parent = Path.cwd()
            cfg.repo = _parent.parent.name if _parent.name=='nbs' else _parent.name
    if lib_path is None: lib_path = cfg.repo.replace('-', '_')
    if copyright is None: copyright = f"{datetime.now().year} onwards, %(author)s"
    for k,v in locals().items():
        if k.startswith('_') or k == 'cfg' or cfg.get(k) is not None: continue
        cfg[k] = v
    return cfg

d### Reading a config file -

Paths are relative to the project:

In [ ]:
# test_eq(cfg.doc_path, p/'_docs')
# test_eq(cfg.lib_path, p/'nbdev')
# test_eq(cfg.nbs_path, p/'nbs')

It automatically returns defaults for keys not specified in the config file. Here we create an empty config file and access `lib_path` and `copyright` even though they weren't explicitly defined: